In [15]:
import os, ast, functools
from collections import defaultdict, Counter
from typing import Optional
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

# ================================================================
# LOAD
# ================================================================
df = pd.read_excel('../data/dimensions/api/raw/combined/202511/df_dimensions.xlsx', index_col=0)

# --------------------------------------------------------------
# SAFE PARSING
# --------------------------------------------------------------
def parse(x):
    if isinstance(x, str):
        try:
            return ast.literal_eval(x)
        except Exception:
            return None
    return x

df["category_for_2020"]  = df["category_for_2020"].apply(parse)
df["researchers"]        = df["researchers"].apply(parse)
df["research_org_names"] = df["research_org_names"].apply(parse)
df["altmetric"]          = pd.to_numeric(df.get("altmetric", 0), errors="coerce").fillna(0)
df["times_cited"]        = pd.to_numeric(df.get("times_cited", 0), errors="coerce").fillna(0)

# ensure lists
df["research_org_names"] = df["research_org_names"].apply(lambda x: x if isinstance(x, list) else [])
df["researchers"]        = df["researchers"].apply(lambda x: x if isinstance(x, list) else [])
df["category_for_2020"]  = df["category_for_2020"].apply(lambda x: x if isinstance(x, list) else [])

# ================================================================
# L4 EXTRACTION (correct, from your working code)
# ================================================================
rows = []

for _, row in df.iterrows():
    cat = row["category_for_2020"]
    if not isinstance(cat, list):
        continue

    L2_list = []
    L4_list = []

    for d in cat:
        nm = d.get("name", "")
        if not nm:
            continue

        parts = nm.split()
        code = parts[0]

        if code.isdigit() and len(code) == 2:
            L2_list.append((code, nm[len(code):].strip()))

        if code.isdigit() and len(code) == 4:
            L4_list.append((code, nm[len(code):].strip()))

    for l2c, l2n in L2_list:
        matched = False
        for l4c, l4n in L4_list:
            if l4c.startswith(l2c):
                matched = True
                rows.append({
                    "id": row["id"],
                    "L2_code": l2c,
                    "L2_name": l2n,
                    "L4_name": l4n,
                    "times_cited": row["times_cited"],
                    "altmetric": row["altmetric"],
                    "researchers": row["researchers"],
                    "orgs": row["research_org_names"]
                })
        if not matched:
            rows.append({
                "id": row["id"],
                "L2_code": l2c,
                "L2_name": l2n,
                "L4_name": None,
                "times_cited": row["times_cited"],
                "altmetric": row["altmetric"],
                "researchers": row["researchers"],
                "orgs": row["research_org_names"]
            })

exploded_L4 = pd.DataFrame(rows)

# ================================================================
# GENDER INFERENCE SETUP (your exact logic)
# ================================================================
import gender_guesser.detector as gender
_DETECTOR1 = gender.Detector(case_sensitive=False)

try:
    from gender_detector.gender_detector import GenderDetector
    _DETECTOR2 = GenderDetector("uk")
    _HAS_DETECTOR2 = True
except Exception:
    _DETECTOR2 = None
    _HAS_DETECTOR2 = False


def _map_gender_guesser(label: Optional[str]) -> str:
    m = (label or "").lower()
    if m in {"male", "mostly_male"}:
        return "male"
    if m in {"female", "mostly_female"}:
        return "female"
    return "unknown"


def _map_gender_detector(label: Optional[str]) -> str:
    l = (label or "").lower()
    return l if l in {"male", "female"} else "unknown"


@functools.lru_cache(maxsize=8192)
def infer_gender_offline(name: Optional[str]) -> str:
    if not isinstance(name, str) or not name.strip():
        return "unknown"
    first = name.split()[0]

    g1 = _map_gender_guesser(_DETECTOR1.get_gender(first))
    if g1 != "unknown":
        return g1

    if _HAS_DETECTOR2 and _DETECTOR2 is not None:
        try:
            g2_raw = _DETECTOR2.guess(first)
        except Exception:
            g2_raw = None
        g2 = _map_gender_detector(g2_raw)
        if g2 != "unknown":
            return g2

    return "unknown"


def extract_forenames(lst):
    out = []
    if not isinstance(lst, list):
        return out
    for r in lst:
        if isinstance(r, dict):
            fn = r.get("first_name", "")
            if fn:
                out.append(fn)
    return out


exploded_L4["forenames"] = exploded_L4["researchers"].apply(extract_forenames)
exploded_L4["gender_list"] = exploded_L4["forenames"].apply(
    lambda xs: [infer_gender_offline(x) for x in xs]
)

# ================================================================
# CALCULATE INSTITUTION-LEVEL percent_female AND modal_L4_name
# ================================================================

# map each institution-paper pair
inst_L4 = []
for _, row in exploded_L4.iterrows():
    if row["orgs"]:
        for inst in row["orgs"]:
            inst_L4.append((inst, row["L4_name"], row["gender_list"]))

inst_L4 = pd.DataFrame(inst_L4, columns=["institution", "L4_name", "gender_list"])

# ---- modal L4 ----
def modal_L4(series):
    vals = [x for x in series if isinstance(x, str)]
    if not vals:
        return None
    return Counter(vals).most_common(1)[0][0]

modal_L4_df = (
    inst_L4.groupby("institution")["L4_name"]
    .apply(modal_L4)
    .reset_index()
    .rename(columns={"L4_name": "modal_L4_name"})
)

# ---- percent female ----
def percent_female_flat(list_of_lists):
    flat = []
    for L in list_of_lists:
        if isinstance(L, list):
            flat.extend(L)
    if not flat:
        return 0.0
    return 100 * sum(g == "female" for g in flat) / len(flat)

percent_female_df = (
    inst_L4.groupby("institution")["gender_list"]
    .apply(percent_female_flat)
    .reset_index()
    .rename(columns={"gender_list": "percent_female"})
)

# ---- total altmetric per institution ----
inst_altmetric = (
    exploded_L4.explode("orgs")
    .groupby("orgs")["altmetric"]
    .sum()
    .reset_index()
    .rename(columns={"orgs": "institution", "altmetric": "total_altmetric"})
)

# ================================================================
# THE ORIGINAL INSTITUTION ANALYSIS (UNCHANGED)
# ================================================================
expanded = (
    df[["id", "times_cited", "research_org_names"]]
    .explode("research_org_names")
    .rename(columns={"research_org_names": "institution"})
)

expanded = expanded.drop_duplicates(subset=["id", "institution"])
total_citations_global = expanded["times_cited"].sum()

def h_index(citations):
    c = sorted(citations, reverse=True)
    return sum(c_i >= (i + 1) for i, c_i in enumerate(c))

def g_index(citations):
    c = sorted(citations, reverse=True)
    s = 0
    g = 0
    for i, c_i in enumerate(c, start=1):
        s += c_i
        if s >= i*i:
            g = i
    return g

def i10_index(citations):
    return sum(c >= 10 for c in citations)

inst_stats = (
    expanded
    .groupby("institution")
    .agg(
        citations_list=("times_cited", lambda x: list(x)),
        papers_in_dataset=("id", "count"),
        unique_papers=("id", "nunique"),
        total_citations=("times_cited", "sum"),
    )
    .reset_index()
)

inst_stats["mean_citations"] = inst_stats["citations_list"].apply(np.mean)
inst_stats["median_citations"] = inst_stats["citations_list"].apply(np.median)
inst_stats["max_citations"] = inst_stats["citations_list"].apply(max)

inst_stats["i10_index"] = inst_stats["citations_list"].apply(i10_index)
inst_stats["h_index_dataset"] = inst_stats["citations_list"].apply(h_index)
inst_stats["g_index_dataset"] = inst_stats["citations_list"].apply(g_index)

inst_stats["h_index_normalised"] = (
    inst_stats["h_index_dataset"] / inst_stats["papers_in_dataset"]
)

inst_stats["citation_variance"] = inst_stats["citations_list"].apply(np.var)
inst_stats["citation_skewness"] = inst_stats["citations_list"].apply(
    lambda c: pd.Series(c).skew()
)

inst_stats["proportion_uncited"] = (
    inst_stats["citations_list"].apply(lambda c: sum(x == 0 for x in c))
    / inst_stats["papers_in_dataset"]
)

inst_stats["citation_share"] = inst_stats["total_citations"] / total_citations_global

# collaboration metrics
paper_inst = df[["id","research_org_names"]].copy()
paper_inst["research_org_names"] = paper_inst["research_org_names"].apply(set)

collab = defaultdict(set)
for _, row in paper_inst.iterrows():
    insts = row["research_org_names"]
    for inst in insts:
        collab[inst].update(insts - {inst})

inst_stats["unique_collaborators"] = inst_stats["institution"].apply(
    lambda inst: len(collab.get(inst, set()))
)

inst_stats["mean_institutions_per_paper"] = inst_stats["institution"].apply(
    lambda inst: (
        df[df["research_org_names"].apply(lambda L: inst in L)]
        ["research_org_names"].apply(len).mean()
    )
)

inst_stats["papers_as_sole_institution"] = inst_stats["institution"].apply(
    lambda inst: (
        df[df["research_org_names"].apply(lambda L: inst in L and len(L)==1)]
        .shape[0]
    )
)

# ================================================================
# MERGE NEW COLUMNS
# ================================================================
inst_stats = inst_stats.merge(inst_altmetric, on="institution", how="left")
inst_stats = inst_stats.merge(percent_female_df, on="institution", how="left")
inst_stats = inst_stats.merge(modal_L4_df, on="institution", how="left")

# ================================================================
# FINAL ORDERING
# ================================================================
institution_summary = inst_stats.sort_values("h_index_dataset", ascending=False)

institution_summary


,institution,citations_list,papers_in_dataset,unique_papers,total_citations,mean_citations,median_citations,max_citations,i10_index,h_index_dataset,...,citation_variance,citation_skewness,proportion_uncited,citation_share,unique_collaborators,mean_institutions_per_paper,papers_as_sole_institution,total_altmetric,percent_female,modal_L4_name
3270,Harvard University,"[125, 15, 116, 1, 0, 1117, 49, 51, 47, 56, 1, ...",1162,1162,118581,102.049053,22.0,2858,800,162,...,64992.757490,5.664480,0.049914,0.014488,3138,17.479346,7,272133.0,26.184505,Genetics
607,Broad Institute,"[232, 42, 9, 10, 33, 160, 30, 56, 181, 189, 68...",678,678,101520,149.734513,41.5,2858,540,152,...,109353.802673,4.740550,0.029499,0.012403,2249,21.407080,0,166499.0,27.670727,Genetics
7529,University of Oxford,"[232, 743, 0, 39, 251, 1117, 49, 85, 14, 160, ...",967,967,96693,99.992761,27.0,2858,680,148,...,52870.145759,5.430378,0.051706,0.011814,2695,16.326784,69,313039.0,31.925788,Clinical Sciences
4670,Massachusetts General Hospital,"[125, 232, 15, 42, 64, 81, 1, 9, 1, 56, 181, 1...",652,652,88014,134.990798,34.0,2858,486,140,...,102461.901756,5.040104,0.032209,0.010753,2440,20.518405,1,161367.0,25.445952,Genetics
7319,University of Cambridge,"[232, 743, 64, 49, 76, 1117, 13, 44, 181, 4, 0...",553,553,61947,112.019892,31.0,2858,419,122,...,53546.392009,5.387341,0.036166,0.007569,2246,20.976492,19,162862.0,30.349325,Genetics
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2813,"FuRong Laboratory, Changsha 410078, China; Hun...",[0],1,1,0,0.000000,0.0,0,0,0,...,0.000000,NaN,1.000000,0.000000,4,5.000000,0,0.0,20.000000,Pollution and Contamination
5861,Radiation Health Center. Liaoning Provincial C...,[0],1,1,0,0.000000,0.0,0,0,0,...,0.000000,NaN,1.000000,0.000000,1,2.000000,0,0.0,12.500000,Clinical Sciences
865,"Center for Mitochondrial Biology and Medicine,...",[0],1,1,0,0.000000,0.0,0,0,0,...,0.000000,NaN,1.000000,0.000000,4,5.000000,0,0.0,10.000000,Medical Biochemistry and Metabolomics
6652,"Strands, 26-32 Pirrama Road, Pyrmont 2009, New...",[0],1,1,0,0.000000,0.0,0,0,0,...,0.000000,NaN,1.000000,0.000000,2,3.000000,0,0.0,0.000000,Genetics


In [19]:
institution_summary.to_excel('../output/tables/institutional_analytics.xlsx')